In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re,string,unicodedata
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,f1_score
from sklearn.model_selection import train_test_split
from string import punctuation
from nltk import pos_tag
from nltk.corpus import wordnet
import re
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt

In [2]:
%%time
df = pd.read_csv(r'C:\Users\Z PC\Documents\sentimental-analysis\IMDB-Dataset.csv')

CPU times: total: 1.14 s
Wall time: 3.87 s


In [3]:
#Customize stopword as per data
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
new_stopwords = ["would","shall","could","might"]
stop_words.extend(new_stopwords)
stop_words.remove("not")
stop_words=set(stop_words)
print(stop_words)

{'how', 'these', 'for', 'does', "won't", 'do', 'only', 'each', 're', "haven't", 'their', 'through', 'some', 't', 'then', 'now', 'wasn', 'shan', "hadn't", 'hasn', 'same', 'out', 'below', 'up', 'don', 'm', 'himself', 'your', 'in', 'doing', 'its', 'into', "weren't", 'a', "isn't", "shan't", 'against', 'didn', 'he', "you'll", 'was', 'until', 'hers', 'myself', 'because', 'from', "mightn't", 'on', 'with', 'about', 'again', 'aren', "it's", 'during', "you've", "you'd", 'been', 'when', "you're", 'o', 'itself', 'of', 'you', 'just', 'should', 'll', 'has', 'once', 's', 'be', 'at', 'those', 'the', 'y', 'no', "that'll", 'this', "wouldn't", 'having', 'am', 'while', 'above', 'my', 'theirs', 'it', 'but', 'our', 'his', 'couldn', 'she', "hasn't", 'isn', 'shouldn', "shouldn't", 'i', 'mightn', 'weren', 'down', 'to', 'that', 'if', 'very', "needn't", 'hadn', 'could', 'both', 'ours', "couldn't", 'wouldn', 'whom', 'her', "should've", 'an', 'will', 'doesn', 'herself', 'd', 'yours', 'them', 'needn', "don't", 'as'

In [4]:
'''-----------------------------Data Cleaning and Preprocessing pipeline----------------------------------'''

#Removing special character
def remove_special_character(content):
    return re.sub('\W+',' ', content )#re.sub('\[[^&@#!]]*\]', '', content)

# Removing URL's
def remove_url(content):
    return re.sub(r'http\S+', '', content)

#Removing the stopwords from text
def remove_stopwords(content):
    clean_data = []
    for i in content.split():
        if i.strip().lower() not in stop_words and i.strip().lower().isalpha():
            clean_data.append(i.strip().lower())
    return " ".join(clean_data)

# Expansion of english contractions
def contraction_expansion(content):
    content = re.sub(r"won\'t", "would not", content)
    content = re.sub(r"can\'t", "can not", content)
    content = re.sub(r"don\'t", "do not", content)
    content = re.sub(r"shouldn\'t", "should not", content)
    content = re.sub(r"needn\'t", "need not", content)
    content = re.sub(r"hasn\'t", "has not", content)
    content = re.sub(r"haven\'t", "have not", content)
    content = re.sub(r"weren\'t", "were not", content)
    content = re.sub(r"mightn\'t", "might not", content)
    content = re.sub(r"didn\'t", "did not", content)
    content = re.sub(r"n\'t", " not", content)
    '''content = re.sub(r"\'re", " are", content)
    content = re.sub(r"\'s", " is", content)
    content = re.sub(r"\'d", " would", content)
    content = re.sub(r"\'ll", " will", content)
    content = re.sub(r"\'t", " not", content)
    content = re.sub(r"\'ve", " have", content)
    content = re.sub(r"\'m", " am", content)'''
    return content

#Data preprocessing
def data_cleaning(content):
    content = contraction_expansion(content)
    content = remove_special_character(content)
    content = remove_url(content)
    
    content = remove_stopwords(content)    
    return content

In [5]:
%%time
pd.options.display.max_colwidth = 1000
#Data cleaning
df['Reviews_clean']=df['Reviews'].apply(data_cleaning)
df.head(5)

CPU times: total: 6.95 s
Wall time: 21 s


,Ratings,Reviews,Movies,Resenhas,Reviews_clean
0,1.0,"*Disclaimer: I only watched this movie as a conditional agreement. And I see films for free. I wouldn't be caught dead giving my hard earned money to these idiots.Well, to explain the depth of this 'film', I could write my shortest review, ever. Don't see this movie. It is by far the stupidest, lamest, most lazy, and unbelievably UNFUNNY movie I have ever seen. It is a total disaster. But since my hatred for this movie, and the others like it, extends far beyond one viewing, I think I'll go on for a bit.I don't know any of the people in the movie besides Carmen Electra, Vanessa Minnillo, and Kim Kardashian, but it doesn't matter. They're all horrible, though I think that was the point. The editing is flat out horrible, and possibly blatant continuity errors make this crapfast even crappier than I thought it would be. Now I know that these films are not supposed to be serious at all, but come on, it's film-making 101 that if someone gets a minor facial cut, it should be there in the...",Disaster Movie,"* Isenção de responsabilidade: eu só assisti esse filme como um acordo condicional. E eu vejo filmes de graça. Eu não seria pego morto dando meu dinheiro suado a esses idiotas. Bem, para explicar a profundidade desse 'filme', eu poderia escrever minha crítica mais curta de todos os tempos. Não vê este filme. É de longe o filme mais estúpido, lamenta, preguiçoso e inacreditavelmente UNFUNNY que eu já vi. É um desastre total. Mas como o meu ódio por este filme e por outros, se estende muito além de uma exibição, acho que vou continuar um pouco. Não conheço nenhuma das pessoas do filme além de Carmen Electra, Vanessa Minnillo, e Kim Kardashian, mas isso não importa. Eles são todos horríveis, embora eu ache que esse seja o ponto. A edição é horrível e, possivelmente, erros de continuidade flagrantes tornam essa porcaria ainda mais horrível do que eu pensava. Agora eu sei que esses filmes não devem ser sérios, mas vamos lá, é o cinema 101 que se alguém fizer um pequeno corte facial, ele...",disclaimer watched movie conditional agreement see films free not caught dead giving hard earned money idiots well explain depth film write shortest review ever not see movie far stupidest lamest lazy unbelievably unfunny movie ever seen total disaster since hatred movie others like extends far beyond one viewing think go bit not know people movie besides carmen electra vanessa minnillo kim kardashian not matter horrible though think point editing flat horrible possibly blatant continuity errors make crapfast even crappier thought know films not supposed serious come film making someone gets minor facial cut next shot someone gets cut sword blood least cut though since narnia films get away give disaster movie pass jokes thoughtless mindless physical gags obviously take popular movies last year late well including best picture nominees know saddest thing stupid movies not care much money make many cameos sorry ass excuses films taking away jobs actors writers directors truly deserv...
1,1.0,"I am writing this in hopes that this gets put over the previous review of this ""film"". How anyone can find this slop entertaining is completely beyond me. First of all a spoof film entitled ""Disaster Movie"", should indeed be a spoof on disaster films. Now I have seen 1 (yes count them, 1) disaster film being spoofed, that being ""Twister"". How does Juno, Iron Man, Batman, The Hulk, Alvin and the Chipmunks, Amy Winehouse, or Hancock register as Disaster films? Selzterwater and Failburg once again have shown that they lack any sort of writing skill and humor. Having unfortunately been tortured with Date Movie and Epic Movie I know exactly what to expect from these two...no plot, no jokes just bad references and cheaply remade scenes from other films. Someone should have informed them that satire is more than just copy and paste from one film to another, though I shouldn't say that because some of these ac

In [6]:
#Mapping rating data to Binary label 1 (+ve) if rating >=7 and 0 (-ve) if rating <=4 and 2 (neutral) if rating = 5 or 6
df['Label'] = df['Ratings'].apply(lambda x: '1' if x >= 7 else ('0' if x<=4 else '2'))
#Removing 
df=df[df.Label<'2']
data=df[['Reviews_clean','Label']]
print(data['Label'].value_counts())

Label
0    60000
1    60000
Name: count, dtype: int64


In [7]:
data.head(5)

,Reviews_clean,Label
0,disclaimer watched movie conditional agreement see films free not caught dead giving hard earned money idiots well explain depth film write shortest review ever not see movie far stupidest lamest lazy unbelievably unfunny movie ever seen total disaster since hatred movie others like extends far beyond one viewing think go bit not know people movie besides carmen electra vanessa minnillo kim kardashian not matter horrible though think point editing flat horrible possibly blatant continuity errors make crapfast even crappier thought know films not supposed serious come film making someone gets minor facial cut next shot someone gets cut sword blood least cut though since narnia films get away give disaster movie pass jokes thoughtless mindless physical gags obviously take popular movies last year late well including best picture nominees know saddest thing stupid movies not care much money make many cameos sorry ass excuses films taking away jobs actors writers directors truly deserv...,0
1,writing hopes gets put previous review film anyone find slop entertaining completely beyond first spoof film entitled disaster movie indeed spoof disaster films seen yes count disaster film spoofed twister juno iron man batman hulk alvin chipmunks amy winehouse hancock register disaster films selzterwater failburg shown lack sort writing skill humor unfortunately tortured date movie epic movie know exactly expect two plot jokes bad references cheaply remade scenes films someone informed satire copy paste one film another though not say actually seem taken trailers nothing clever witty remotely smart way two write not believe people still pay see travesties insult audience though enjoy films doubt smart enough realize rating unfortunately not number low enough yes includes negatives rate deserves top worst films time right date movie epic faliure mean movie meet spartans rather forced hour manos hands fate marathon watch slop,0
2,really write scathing review turd sandwich instead going making observations points deduced point watching movies anymore reader remember scary movie remember original comedic elements slapstick funny lines pretty forgettable comedy worth price admission well last time premise funny stop making movies please call boycott pieces monkey sh know going line pre pubescent annoying little buggers spouting crappy one liners like sparta im rick james bitch movies continue make form monetary gain considering production value movie looks like cost cents make not see movie not spend money go home rent airplane laugh ass silently judge people talking movie monday favor,0
3,saw previous spoof movies two horrible gentlemen know already bad tell truth want watch brainless person ironically meant stereotypical teenagers not laugh bit judge even little movie automatically fails never ask comes two men remember good old hollywood days whenever making movie showing people type art also story kept edge seat well whenever word hit making films earned loads cash greedy people came picture quite pathetic two exception still movie artists notably genius christopher nolan two guys well writing big words let put simple terms guys guys suck not artists instead money craving whores latest movie proves even movie fails easily mind blowing mean nothing funny trailer people usually put best stuff like idiots sometimes knew going bad made bet friends not good idea write movie reviews paper tell everyone whats good whats bad friends flipped review well warning least not even called movie nothing artistic original jokes sorry references made throughout pretty much random ...,0
4,movie saw day early free still feel like got ripped totally brain dead burping kicking groin boobs place lame wrong society films like even get made parodies horrendous un funny plot lackluster best acting shallow transparent really quite unnecessary anyone see idiocracy remember movie academy awards future well movie not seen rancid crappy film date mov

In [8]:
data.tail()

,Reviews_clean,Label
149995,goldeneye number personal favorite movie time james bond action film best one ever made opinion best one love movie death grew watching child pierce brosnan number favorite james bond goldeneye first film star pierce brosnan fictional officer james bond film directed martin campbell first series not take story elements works novelist ian fleming hard core james bond fan make apologies believing pierce brosnan closest thing seen ian fleming james bond reason favorite feel action action action action never feel bored long like bond movies fast paced entertaining hard core action plot simply story never get bored beautiful movie beautiful direction martin campbell stunts completely insane scale movie really dangerous music score éric serra title song goldeneye performed tina turner goldeneye released six year hiatus series caused legal disputes timothy dalton resigned role james bond replaced pierce brosnan also recast actress judi dench becoming first woman portray character replacin...,1
149996,pierce brosnan izabella scorupco sean bean famke janssen double agent james bond still suave sophisticated last film enters russia search stolen goldeneye satellite six years without fans needed movie like everything works witty plot pierce brosnan new bond not review without complimenting sean bean series best villain rank series,1
149997,given film not thoroughly enjoyed believe best bond ever made actually first one watched well watched seen rest since still one favourite pierce brosnan terrific first appearance bond think connery best stepped role far izabella scorupco gorgeous sidekick bond girl famke janssen brilliant sexy villain partnering sean bean whose also excellent plot simple interesting importantly action sequences truly outstanding opening minutes definitely right best bond openings probably best movie never drags like several others wish action get going action sequences decent include good aircraft crashes explosions satellite dish great shootout military archive leads onto highlight movie outstanding tank chase russia climax also great well set top strange scaffolding features great suspenseful fight brosnan penn film directed martin campbell must one underrated action filmmakers around days think debut possibly first proper hollywood blockbuster certainly makes opportunity given also liked film ve...,1
149998,absolutely adore movie comeback bond timothy dalton bond franchise officially declared death many one expected another bond movie movie perfectly sets mood future bond movies decade action makes movie way time way bond franchise matured became competitor box office still old bond elements present movie still stays loyal franchise many grew watching villains best ever bond movie since goldfinger funny movie also opinion best bond movie since goldfinger think tells something important bond villains always bond movies sean bean memorable worthy nemesis bond alan cumming excellent fun villain memorable famke janssen literally deathliest hips hit screen movie really launched career three actors coincidence think pierce brosnan simply bond moment first see got charms humor two important elements bond character movie introduces nice new characters later return bond movies judi dench plays new joe baker plays fun character jack wade robbie coltrane plays highly enjoyable villain valentin z...,1
149999,rented boogie nights last week tell watched film blast think watch film get sicked porn mean yes not porn person not bother not film see thing whole film not really porn well halfway film porn industry half character development bad situations characters go actors played roles perfect especially mark wahlberg john c reilly william h macy sex scenes course terrific mainly focus character hype porn films struggles excellent film one best hedeen outlook,1


In [9]:
#Importing dependencies for feature engineering 
import sys
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from prettytable import PrettyTable
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer

                Lemmataizaiton

In [10]:
# lemmatization of word 
class LemmaTokenizer(object):
    def __init__(self):
        self.wordnetlemma = WordNetLemmatizer()
    def __call__(self, reviews):
        return [self.wordnetlemma.lemmatize(word) for word in word_tokenize(reviews)]

In [24]:
import nltk
nltk.data.path.append("C:\\Users\\Z PC\\AppData\\Roaming\\nltk_data")


In [21]:
import nltk
print(nltk.data.path)


['C:\\Users\\Z PC/nltk_data', 'c:\\Users\\Z PC\\Documents\\sentimental-analysis\\venv\\nltk_data', 'c:\\Users\\Z PC\\Documents\\sentimental-analysis\\venv\\share\\nltk_data', 'c:\\Users\\Z PC\\Documents\\sentimental-analysis\\venv\\lib\\nltk_data', 'C:\\Users\\Z PC\\AppData\\Roaming\\nltk_data', 'C:\\nltk_data', 'D:\\nltk_data', 'E:\\nltk_data']


Vectoization with Count Vectorizer and TDIDF Vectorizer with Unigram

In [25]:
from nltk.tokenize import word_tokenize
print(word_tokenize("This is a test sentence."))


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - 'C:\\Users\\Z PC/nltk_data'
    - 'c:\\Users\\Z PC\\Documents\\sentimental-analysis\\venv\\nltk_data'
    - 'c:\\Users\\Z PC\\Documents\\sentimental-analysis\\venv\\share\\nltk_data'
    - 'c:\\Users\\Z PC\\Documents\\sentimental-analysis\\venv\\lib\\nltk_data'
    - 'C:\\Users\\Z PC\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - 'C:\\Users\\Z PC\\AppData\\Roaming\\nltk_data'
**********************************************************************


In [26]:
import nltk
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt_tab to C:\Users\Z
[nltk_data]     PC\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


True

In [27]:
from nltk.tokenize import word_tokenize
print(word_tokenize("This is a test sentence."))


['This', 'is', 'a', 'test', 'sentence', '.']


In [29]:
import nltk
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to C:\Users\Z
[nltk_data]     PC\AppData\Roaming\nltk_data...


True

In [30]:
train,test=train_test_split(data,test_size=.3,random_state=42, shuffle=True)
countvect = CountVectorizer(analyzer = "word", tokenizer = LemmaTokenizer(), ngram_range=(1,1), min_df=10,max_features=500)
tfidfvect = TfidfVectorizer(analyzer = "word", tokenizer = LemmaTokenizer(), ngram_range=(1,1),min_df=10,max_features=500)
x_train_count = countvect.fit_transform(train['Reviews_clean']).toarray()
x_test_count = countvect.transform(test['Reviews_clean']).toarray()
x_train_tfidf = tfidfvect.fit_transform(train['Reviews_clean']).toarray()
x_test_tfidf = tfidfvect.transform(test['Reviews_clean']).toarray()
y_train = train['Label']
y_test = test['Label']

Feature Importance with Logistic Regression and Count Vectorizer with unigram